<a href="https://colab.research.google.com/github/syedmahmoodiagents/Speech/blob/main/ASR_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchcodec --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torchaudio import load
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [ ]:
from pydub import AudioSegment
from pydub.playback import play

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
import os
os.makedirs('./data', exist_ok=True)
dataset = torchaudio.datasets.SPEECHCOMMANDS("./data", download=True)

100%|██████████| 2.26G/2.26G [00:33<00:00, 73.0MB/s]


In [ ]:
dataset.get_metadata(-0)

('speech_commands_v0.02/backward/0165e0e8_nohash_0.wav',
 16000,
 'backward',
 '0165e0e8',
 0)

In [ ]:
len(dataset)

105829

In [ ]:
dataset[13002]

(tensor([[0.0019, 0.0035, 0.0047,  ..., 0.0049, 0.0056, 0.0048]]),
 16000,
 'down',
 'cc71bada',
 0)

In [ ]:
dataset[13002][0]

tensor([[0.0019, 0.0035, 0.0047,  ..., 0.0049, 0.0056, 0.0048]])

In [ ]:
audio_tensor = dataset[13002][0]

In [ ]:

torchaudio.save('sample_audio.wav', audio_tensor, sample_rate=16000)

In [ ]:
torchaudio.save('sample_audio.mp3', audio_tensor, sample_rate=16000)

In [ ]:
# audio = AudioSegment.from_file('sample_audio.wav')

In [ ]:
# audio.export('sample_audio.mp3', format='mp3')

In [ ]:
AudioSegment.from_file('sample_audio.mp3')

# Shorter Database

In [ ]:
import os, glob

root = "./data/SpeechCommands/speech_commands_v0.02/"

In [ ]:
labels = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]

print(labels)
print("Number of labels =", len(labels))


['backward', 'marvin', 'eight', 'right', '_background_noise_', 'go', 'yes', 'four', 'forward', 'happy', 'left', 'two', 'visual', 'one', 'seven', 'house', 'tree', 'on', 'down', 'off', 'nine', 'five', 'stop', 'six', 'bed', 'zero', 'bird', 'dog', 'sheila', 'cat', 'wow', 'three', 'learn', 'up', 'no', 'follow']
Number of labels = 36


In [ ]:
import os, glob

In [ ]:
glob.glob(os.path.join('sample_data','SecondFolder', "*.txt"))

['sample_data/SecondFolder/abcd.txt', 'sample_data/SecondFolder/xyz.txt']

In [ ]:
samples_per_label = 5
selected_files = {}
for label in labels:
    wavs = glob.glob(os.path.join(root, label, "*.wav"))
    selected_files[label] = wavs[:5]  # first 5 files


In [ ]:
print("cat:", selected_files["cat"])
print("bird:", selected_files["bird"])
print("dog:", selected_files["dog"])

cat: ['./data/SpeechCommands/speech_commands_v0.02/cat/324210dd_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/cat/0cb74144_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/cat/53578f4e_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/cat/7846fd85_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/cat/afb9e62e_nohash_0.wav']
bird: ['./data/SpeechCommands/speech_commands_v0.02/bird/324210dd_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/bird/0cb74144_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/bird/53578f4e_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/bird/7846fd85_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/bird/afb9e62e_nohash_0.wav']
dog: ['./data/SpeechCommands/speech_commands_v0.02/dog/324210dd_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/dog/0cb74144_nohash_0.wav', './data/SpeechCommands/speech_commands_v0.02/dog/53578f4e_nohash_0.wav', './data/SpeechCommands/spee

In [ ]:
labels

['backward',
 'marvin',
 'eight',
 'right',
 '_background_noise_',
 'go',
 'yes',
 'four',
 'forward',
 'happy',
 'left',
 'two',
 'visual',
 'one',
 'seven',
 'house',
 'tree',
 'on',
 'down',
 'off',
 'nine',
 'five',
 'stop',
 'six',
 'bed',
 'zero',
 'bird',
 'dog',
 'sheila',
 'cat',
 'wow',
 'three',
 'learn',
 'up',
 'no',
 'follow']

In [ ]:
class FewShotSCFromFiles(Dataset):
    def __init__(self, file_dict):
        self.items = []
        for label, files in file_dict.items():
            for f in files:
                self.items.append((f, label))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, label = self.items[idx]
        audio, sr = load(path)
        return audio, sr, label



In [ ]:
fewshot = FewShotSCFromFiles(selected_files)

In [ ]:

print("dataset size:", len(fewshot))

dataset size: 180


In [ ]:
fewshot.__len__()

180

In [ ]:
fewshot[60]

(tensor([[ 3.0518e-05,  6.1035e-05,  6.1035e-05,  ..., -1.5259e-04,
          -1.8311e-04, -1.8311e-04]]),
 16000,
 'visual')

In [ ]:
fewshot.__getitem__(60)

(tensor([[ 3.0518e-05,  6.1035e-05,  6.1035e-05,  ..., -1.5259e-04,
          -1.8311e-04, -1.8311e-04]]),
 16000,
 'visual')

## New Dataset

In [ ]:
class FewShotMFCCDataset(Dataset):
    def __init__(self, file_dict, labels, n_mfcc=40):
        self.items = []
        self.labels = labels
        self.label_to_idx = {l:i for i,l in enumerate(labels)}

        for label, files in file_dict.items():
            for f in files:
                self.items.append((f, label))

        self.mfcc = torchaudio.transforms.MFCC(sample_rate=16000,n_mfcc=n_mfcc)
        self.max_len = 16000  # 1 sec

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, label = self.items[idx]
        audio, sr = torchaudio.load(path)


        if audio.shape[0] > 1:
            audio = audio.mean(dim=0, keepdim=True)


        if sr != 16000:
            audio = torchaudio.functional.resample(audio, sr, 16000)


        if audio.shape[1] < self.max_len:
            audio = F.pad(audio, (0, self.max_len - audio.shape[1]))
        else:
            audio = audio[:, :self.max_len]


        mfcc = self.mfcc(audio)   # [1, n_mfcc, time]
        mfcc = mfcc.squeeze(0)    # [n_mfcc, time]


        x = mfcc.flatten()

        y = self.label_to_idx[label]

        return x, y


In [ ]:
newdata = FewShotMFCCDataset(selected_files, labels)

/usr/local/lib/python3.12/dist-packages/torchaudio/functional/functional.py:582: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [ ]:
newdata[0][0].shape

torch.Size([3240])

In [ ]:
input_dim = newdata[0][0].shape[0]

In [ ]:
loader = DataLoader(newdata, batch_size=16, shuffle=True)

In [ ]:
print("Input vector size =", input_dim)
print("Dataset size =", len(newdata))

Input vector size = 3240
Dataset size = 180


In [ ]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.net(x)


In [ ]:
model = MLP(input_dim, len(labels))

In [ ]:
import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(20):
    model.train()
    correct = 0
    total = 0

    for x, y in loader:
        outputs = model(x)
        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    print(f"Epoch {epoch+1}: Accuracy = {correct/total:.3f}")


Epoch 1: Accuracy = 0.106
Epoch 2: Accuracy = 0.156
Epoch 3: Accuracy = 0.394
Epoch 4: Accuracy = 0.639
Epoch 5: Accuracy = 0.800
Epoch 6: Accuracy = 0.928
Epoch 7: Accuracy = 0.961
Epoch 8: Accuracy = 0.967
Epoch 9: Accuracy = 0.994
Epoch 10: Accuracy = 1.000
Epoch 11: Accuracy = 1.000
Epoch 12: Accuracy = 1.000
Epoch 13: Accuracy = 1.000
Epoch 14: Accuracy = 1.000
Epoch 15: Accuracy = 1.000
Epoch 16: Accuracy = 1.000
Epoch 17: Accuracy = 1.000
Epoch 18: Accuracy = 1.000
Epoch 19: Accuracy = 1.000
Epoch 20: Accuracy = 1.000


In [ ]:
idx_to_label = {i:l for l,i in newdata.label_to_idx.items()}

In [ ]:
def preprocess_audio(path, mfcc_transform, max_len=16000):
    audio, sr = torchaudio.load(path)

    # mono
    if audio.shape[0] > 1:
        audio = audio.mean(dim=0, keepdim=True)

    # resample
    if sr != 16000:
        audio = torchaudio.functional.resample(audio, sr, 16000)

    # pad/truncate to 1 sec
    if audio.shape[1] < max_len:
        audio = F.pad(audio, (0, max_len - audio.shape[1]))
    else:
        audio = audio[:, :max_len]

    # MFCC
    mfcc = mfcc_transform(audio)   # [1, n_mfcc, time]
    mfcc = mfcc.squeeze(0)         # [n_mfcc, time]

    # flatten
    x = mfcc.flatten().unsqueeze(0)  # shape [1, INPUT_DIM]

    return x


In [ ]:
def predict_audio(path, model, mfcc_transform, idx_to_label):
    x = preprocess_audio(path, mfcc_transform)
    with torch.no_grad():
        logits = model(x)
        pred_idx = logits.argmax(dim=1).item()
        return idx_to_label[pred_idx]


In [ ]:
mfcc_transform = newdata.mfcc  # reuse same MFCC


In [ ]:
idx_to_label = {v:k for k,v in newdata.label_to_idx.items()}

In [ ]:
result = predict_audio("data/SpeechCommands/speech_commands_v0.02/backward/02ade946_nohash_0.wav", model, mfcc_transform, idx_to_label)

In [ ]:
result

'seven'